In [11]:
from v_diffusion import *
import torch
from torchvision.utils import save_image

model = UNet(
    in_channels=3,
    hid_channels=64,
    out_channels=6,
    ch_multipliers=[1, 2, 3, 4],
    num_res_blocks=2,
    apply_attn=[False, True, True, True],
    drop_rate=0.2,
    #num_classes=10
)

device = "cuda:0"
chkpt = torch.load("chkpts/vdpm_lsun_2.pt", map_location=device)["ema"]["shadow"]
model.to(device)
for k in list(chkpt.keys()):
    if k.startswith("module."):
        chkpt[k.split(".", maxsplit=1)[1]] = chkpt.pop(k)
model.load_state_dict(chkpt)
model.eval()
model.requires_grad_(False)
del chkpt

logsnr_fn = get_logsnr_schedule("cosine", -20, 20)
diffusion = GaussianDiffusion(
    logsnr_fn,
    sample_timesteps=128,
    model_out_type="both",
    model_var_type="fixed_large",
    reweight_type="truncated_snr",
    loss_type="mse",
    w_guide=0.1,
    p_uncond=0.1)


In [26]:

ws = [3]
# torch.manual_seed(1234)
noise = torch.randn((1, 3, 64, 64)).to(device)

for w in ws:
    diffusion.w_guide = w

    x_gen = diffusion.p_sample(
        model, (1, 3, 64, 64), device=device,
        noise=noise,
        # label=torch.arange(
        #     1, 11, device=device, dtype=torch.float32).repeat_interleave(10),
        # label=torch.tensor(
        #     0, device=device, dtype=torch.float32),
        label=None,
        use_ddim=True)

    save_image(
        x_gen.cpu(),
        f"./cifar10_w{w}.png",
        nrow=1,
        normalize=True,
        value_range=(-1, 1)
    )